In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import dateparser
import pandas as pd
import json
import nltk
import numpy as np
import string
import re
import csv
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
from functions import *
import dateparser
from fuzzywuzzy import process
from ccfMitchell import ccf

In [2]:
import stanza

In [3]:
df_data = load_data_tagtog(['../tagtog/output']).fillna('')
df_data = df_data.loc[df_data['is_flood'].astype(bool)]

In [4]:
# 2723 - old data
# 3464 - new data
# 3074 - new new data
override=False
if not override and os.path.exists('timeseries_data/all_data/all_isFlood.json'):
    df_data = pd.read_json('timeseries_data/all_data/all_isFlood.json')
else:
    df_data = add_prev_true_data(df_data)
    df_data = add_newspapers(df_data)
    df_data = add_datePublished(df_data)
    df_data = add_location(df_data)
    df_data = df_data[df_data['divisions'].apply(lambda x:len(x)>0)]
    json.dump(json.loads(df_data.to_json(orient='records')), open('timeseries_data/all_data/all_isFlood.json','w'), indent=2)
df_data.head()

,doc_id,filename,text,is_flood,is_bangladesh,flood_related,flood_climatechange,newspaper,flood_type,dates,...,event_damage-people_affected,event_damage-peopled_displaced,event_damage-homes_affected,event_damage-disease,event_damage-fatalities,event_dates,event_dates-date,event_dates-prev_date,datePublished,connect_filename
0,ec583817-3c60-41ee-b856-65f0d9bd7772,dailySun_data_ec583817-3c60-41ee-b856-65f0d9bd...,Date Published:2017-08-31 06:03:11+00:00 tuesd...,True,,,,dailySun,,[],...,[],[],[],[],[],[],[],[],2017-08-31 06:03:11+00:00,
1,f4806621-b874-4f20-97fb-f7c1fa94f6bc,theDailyStar_data_f4806621-b874-4f20-97fb-f7c1...,Date Published:2016-05-05 00:00:00 Flash flood...,True,True,True,False,theDailyStar,flash,[],...,[],[],[],[],[],[],[],[],2016-05-05 00:00:00,
2,259d503d-f6b1-44b6-a866-8eff03799a07,prothomalo_data_259d503d-f6b1-44b6-a866-8eff03...,Date Published:None Flood situation worsened f...,True,,,,prothomalo,,[],...,[],[],[],[],[],[],[],[],2019-07-17T03:35:13Z,
3,1e8c824c-6142-496e-ab4b-aeefaeedc681,theIndependent_data_1e8c824c-6142-496e-ab4b-ae...,Date Published:2016-06-04 10:27:51 Rail commun...,True,,,,theIndependent,,[],...,[],[],[],[],[],[],[],[],2016-06-04 10:27:51,
4,7c2bc791-c162-46bf-9224-2686d6798c33,dailySun_data_7c2bc791-c162-46bf-9224-2686d679...,Date Published:2019-09-30 13:44:33+00:00 tuesd...,True,,,,dailySun,,[],...,[],[],[],[],[],[],[],[],2019-09-30 13:44:33+00:00,


In [5]:
dates_all = query_dataframe(df_data,{'is_flood':True})['datePublished']
dates = parse_all_dates(dates_all)
df_data['datesP'] = dates

('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', '2031-12-30 00:00:00') 2031-12-30 00:00:00
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Present', 'None') None
('No Date Pre

In [6]:
datesLocation = defaultdict(set)
for row in df_data.iterrows():
    entry = row[1]
    dates = entry['datesP']
    if not dates:
#         print(entry)
        continue
    year, month, day = dates['year'], dates['month'], dates['day']
    if not year or not month or not day:
#         print(entry)
        continue
    for loc in entry['districts']: datesLocation['{}-{}-{}'.format(year, month, day)].add(loc)

### Get location entires

In [7]:
import requests
import json 
import xmltodict

In [50]:
# def extract_loc_geonames(d):
#     return {entry['geonameId']:entry['name'] 
#              for entry in d['geonames']['geoname']}

#### Divisions

In [51]:
# r = requests.get('http://api.geonames.org/children?geonameId=1210997&username=tvp2107')
# res = xmltodict.parse(r.text)
# divisions = extract_loc_geonames(res)

#### Districts

In [56]:
# div_to_dist = {}
# for geoID,name in divisions.items():
#     r = requests.get('http://api.geonames.org/children?geonameId={}&username=tvp2107'.format(geoID))
#     res = xmltodict.parse(r.text)
#     div_to_dist[name] = extract_loc_geonames(res)

#### Upazilla

In [60]:
# dist_to_up = {}
# for _,distD in div_to_dist.items():
#     for geoID,name in distD.items():
#         r = requests.get('http://api.geonames.org/children?geonameId={}&username=tvp2107'.format(geoID))
#         res = xmltodict.parse(r.text)
#         dist_to_up[name] = extract_loc_geonames(res)

#### Save

In [63]:
# format_path = 'timeseries_data/logistics/geeonames_{}.json'
# json.dump(divisions, open(format_path.format('division'),'w'), indent=2)
# json.dump(div_to_dist, open(format_path.format('div_dist'),'w'), indent=2)
# json.dump(dist_to_up, open(format_path.format('dist_up'),'w'), indent=2)

#### Load

In [8]:
format_path = 'timeseries_data/logistics/geeonames_{}.json'
divisions = json.load(open(format_path.format('division'),'r'))
div_to_dist = json.load(open(format_path.format('div_dist'),'r'))
dist_to_up = json.load(open(format_path.format('dist_up'),'r'))
districts = [dist for _,distD in div_to_dist.items() 
                 for _,dist in distD.items()]
upazillas = [upa for _,upaD in dist_to_up.items() 
                 for _,upa in upaD.items()]

### Location

In [9]:
nlp = stanza.Pipeline('en')

2021-03-15 22:36:49 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-03-15 22:36:49 INFO: Use device: cpu
2021-03-15 22:36:49 INFO: Loading: tokenize
2021-03-15 22:36:49 INFO: Loading: pos
2021-03-15 22:36:50 INFO: Loading: lemma
2021-03-15 22:36:50 INFO: Loading: depparse
2021-03-15 22:36:50 INFO: Loading: sentiment
2021-03-15 22:36:51 INFO: Loading: ner
2021-03-15 22:36:52 INFO: Done loading processors!


In [12]:
loc_arr = {}
filepath = 'timeseries_data/logistics/docid_locations.json'
for i,row in enumerate(df_data.iterrows()):
    text = row[1]['text']
    doc_id = row[1]['doc_id']
    doc = nlp(text)
    arr = set()
    try:
        for d in doc.entities:
            try:
                if d.type == "GPE":
                    arr.add(d.text)
            except Exception as e:
                print(d)
                print(e)
                print('------------')
                continue
    except Exception as e:
        print(i)
        print(e)
        print('------------')
        loc_arr[doc_id] = list(arr)
        continue
    loc_arr[doc_id] = list(arr)
    if i%100==0:
        json.dump(loc_arr, open(filepath,'w'), indent=2)
        print('.', end='')
json.dump(loc_arr, open(filepath,'w'), indent=2)

.......................................

In [44]:
choices = list(divisions.values()) + districts + upazillas
choices = [c.lower() for c in choices]
replace_items = ['village', 'upazila', 'division', 'district']
replace_items += [r+'s' for r in replace_items]
loc_arr_search = []
for i,row in enumerate(df_data.iterrows()):
    text = row[1]['text']
    doc_id = row[1]['doc_id']
    if not doc_id:
        loc_arr_search.append([])
    else:
        entry = loc_arr[doc_id]
        arr = []
        for loc in entry:
            loc = loc.lower()
            for r in replace_items: loc = loc.replace(r, '')
            loc_f = process.extract(loc.lower().strip(), choices, limit=1)
            arr.append(loc_f[0][0])
        loc_arr_search.append(arr)
    if i%100==0:
        print('.', end='')

.

.

.

.

.

..................................

In [53]:
dist_to_div, up_to_dist = {}, {}

In [55]:
for div,v in div_to_dist.items():
    for _,dist in v.items():
        dist_to_div[dist.lower()] = div.lower()
for dist,v in dist_to_up.items():
    for _,upa in v.items():
        up_to_dist[upa.lower()] = dist.lower()

In [63]:
divisions_ner, districts_ner, upazilla_ner = [], [], []
for entry in loc_arr_search:
    div_inner, dist_inner, upa_inner = set(), set(), set()
    for r in entry:
        r = r.lower()
        if r in up_to_dist:
            upa_inner.add(r)
            dist_inner.add(up_to_dist[r])
            div_inner.add(dist_to_div[up_to_dist[r]])
        elif r in dist_to_div:
            dist_inner.add(r)
            div_inner.add(dist_to_div[r])
        elif r in divisions:
            div_inner.add(r)
    divisions_ner.append(list(div_inner))
    districts_ner.append(list(dist_inner))
    upazilla_ner.append(list(upa_inner))

In [67]:
df_data['divisions_ner'] = divisions_ner
df_data['districts_ner'] = districts_ner
df_data['upazilla_ner'] = upazilla_ner

In [ ]:
df_data['divisions_comb'] = 